In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
import json

class Model:
    def __init__(self, threshold=0.1, n_clusters=3, random_state=42):
        """
        Класс для анализа метрик успешных и неуспешных спринтов.
        :param threshold: Процентное отношение между метриками, чтобы считать их выделяющимися.
        :param n_clusters: Количество кластеров для кластеризации.
        :param random_state: Случайное состояние для воспроизводимости.
        """
        self.threshold = threshold
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.scaler = StandardScaler()
        self.kmeans_successful = MiniBatchKMeans(n_clusters=self.n_clusters, random_state=self.random_state)
        self.kmeans_unsuccessful = MiniBatchKMeans(n_clusters=self.n_clusters, random_state=self.random_state)

    def select_significant_metrics(self, metric_series):
        """
        Выбирает метрики, которые значительно отличаются от остальных.
        :param metric_series: Series с диапазонами метрик.
        :return: Список метрик.
        """
        sorted_metrics = metric_series.sort_values(ascending=False)
        max_value = sorted_metrics.iloc[0]
        significant_metrics = sorted_metrics[sorted_metrics >= max_value * (1 - self.threshold)]
        return significant_metrics.index.tolist()

    def cluster_data(self, successful_data, unsuccessful_data):
        """
        Проводит кластеризацию успешных и неуспешных данных.
        :param successful_data: DataFrame с успешными спринтами.
        :param unsuccessful_data: DataFrame с неуспешными спринтами.
        :return: Два DataFrame с добавленными кластерами.
        """
        # Масштабирование данных
        features_successful = successful_data.drop(columns=["sprint_id", "success"])
        features_unsuccessful = unsuccessful_data.drop(columns=["sprint_id", "success"])
        scaled_successful = self.scaler.fit_transform(features_successful)
        scaled_unsuccessful = self.scaler.fit_transform(features_unsuccessful)

        # Кластеризация
        successful_clusters = self.kmeans_successful.fit_predict(scaled_successful)
        unsuccessful_clusters = self.kmeans_unsuccessful.fit_predict(scaled_unsuccessful)

        # Добавление кластеров в исходные данные
        successful_data = successful_data.copy()
        unsuccessful_data = unsuccessful_data.copy()
        successful_data["cluster"] = successful_clusters
        unsuccessful_data["cluster"] = unsuccessful_clusters

        return successful_data, unsuccessful_data

    def analyze_clusters(self, successful_data, unsuccessful_data):
        """
        Проводит анализ метрик кластеров для успешных и неуспешных данных.
        :param successful_data: DataFrame с кластерами успешных спринтов.
        :param unsuccessful_data: DataFrame с кластерами неуспешных спринтов.
        :return: Анализ кластеров и наиболее значимые метрики.
        """
        # Расчет средних значений и вариативности для каждого кластера
        successful_cluster_means = successful_data.groupby("cluster").mean()
        unsuccessful_cluster_means = unsuccessful_data.groupby("cluster").mean()
        successful_cluster_variability = successful_data.groupby("cluster").std()
        unsuccessful_cluster_variability = unsuccessful_data.groupby("cluster").std()

        # Различия между кластерами
        successful_diff = successful_cluster_means.max() - successful_cluster_means.min()
        unsuccessful_diff = unsuccessful_cluster_means.max() - unsuccessful_cluster_means.min()

        # Формирование итогового анализа
        cluster_analysis = {
            "Successful Cluster Mean Range": successful_diff,
            "Successful Cluster Variability": successful_cluster_variability.mean(),
            "Unsuccessful Cluster Mean Range": unsuccessful_diff,
            "Unsuccessful Cluster Variability": unsuccessful_cluster_variability.mean()
        }

        # Выбор наиболее выделяющихся метрик
        successful_top_metrics = self.select_significant_metrics(successful_diff)
        unsuccessful_top_metrics = self.select_significant_metrics(unsuccessful_diff)

        # Результат анализа
        highlighted_metrics = {
            "successful_sprints": successful_top_metrics,
            "unsuccessful_sprints": unsuccessful_top_metrics
        }

        return cluster_analysis, highlighted_metrics

    def load_data_from_json(self, json_data: dict) -> pd.DataFrame:
        self.df = pd.DataFrame(json_data["data"])
        if self.df.empty:
            raise ValueError("Loaded data is empty.")

        print("Data loaded successfully from JSON.")

        return self.df

    def save_to_json(self, data, file_name="highlighted_metrics.json"):
        """
        Сохраняет данные в JSON-файл.
        :param data: Словарь с данными для сохранения.
        :param file_name: Имя файла для сохранения.
        """
        with open(file_name, "w") as file:
            json.dump(data, file)

# Пример использования класса
# model = Model(threshold=0.1, n_clusters=3)
# cluster_analysis, result = model.cluster_and_analyze(successful_sprints, unsuccessful_sprints)
# model.save_to_json(result, "highlighted_metrics.json")


In [11]:
np.random.seed(42)

# Количество записей
n_samples = 500

# Пример метрик спринта
data = {
    "sprint_id": range(1, n_samples + 1),
    "tasks_completed": np.random.randint(5, 50, n_samples),  # Количество завершенных задач
    "tasks_planned": np.random.randint(30, 60, n_samples),   # Количество запланированных задач
    "team_velocity": np.random.uniform(20, 100, n_samples),  # Скорость команды
    "bugs_reported": np.random.randint(0, 10, n_samples),    # Количество найденных багов
    "team_satisfaction": np.random.uniform(3, 10, n_samples), # Уровень удовлетворенности команды (1-10)
    "scope_changes": np.random.randint(0, 5, n_samples),     # Количество изменений в объеме задач
    "success": np.random.choice([0, 1], n_samples)           # Метка успешности спринта (0 - неуспешный, 1 - успешный)
}

# Создание DataFrame
def load_data_from_json(self, json_data: dict) -> pd.DataFrame:
        self.df = pd.DataFrame(json_data["data"])
        if self.df.empty:
            raise ValueError("Loaded data is empty.")

        print("Data loaded successfully from JSON.")

        return self.df 
df = pd.DataFrame(data)

# Добавление метрики процента завершенных задач (derived feature)
df["completion_rate"] = (df["tasks_completed"] / df["tasks_planned"]).clip(0, 1)

# Разделение данных на успешные и неуспешные спринты
successful_sprints = df[df["success"] == 1].reset_index(drop=True)
unsuccessful_sprints = df[df["success"] == 0].reset_index(drop=True)


# Отображение пользователю
# import ace_tools as tools; tools.display_dataframe_to_user(name="Успешные спринты", dataframe=successful_sprints)
# tools.display_dataframe_to_user(name="Неуспешные спринты", dataframe=unsuccessful_sprints)


In [8]:
successful_sprints.head()

,sprint_id,tasks_completed,tasks_planned,team_velocity,bugs_reported,team_satisfaction,scope_changes,completion_rate
0,1,43,52,50.182733,2,4.484314,1,0.826923
1,2,33,46,21.605696,9,7.276867,1,0.717391
2,5,12,58,46.199788,7,9.300162,4,0.206897
3,8,23,55,67.487396,0,8.463679,2,0.418182
4,10,15,31,83.133699,7,8.758300,2,0.483871


In [9]:
unsuccessful_sprints.head(
    
)

,sprint_id,tasks_completed,tasks_planned,team_velocity,bugs_reported,team_satisfaction,scope_changes,completion_rate
0,3,19,55,45.766333,2,5.877199,1,0.345455
1,4,47,37,36.915841,7,8.879029,2,1.000000
2,6,25,55,29.580971,1,5.473950,3,0.454545
3,7,43,39,91.242182,7,4.658094,3,1.000000
4,9,27,59,74.328186,5,4.923642,1,0.457627


In [12]:


# Масштабирование данных
scaler = StandardScaler()

# Выбираем только числовые столбцы для кластеризации (исключая метки и идентификаторы)
features_successful = successful_sprints.drop(columns=["sprint_id", "success"])
features_unsuccessful = unsuccessful_sprints.drop(columns=["sprint_id", "success"])

# Масштабируем данные
scaled_successful = scaler.fit_transform(features_successful)
scaled_unsuccessful = scaler.fit_transform(features_unsuccessful)

# Инициализация моделей MiniBatchKMeans
kmeans_successful = MiniBatchKMeans(n_clusters=3, random_state=42)
kmeans_unsuccessful = MiniBatchKMeans(n_clusters=3, random_state=42)

# Кластеризация
successful_clusters = kmeans_successful.fit_predict(scaled_successful)
unsuccessful_clusters = kmeans_unsuccessful.fit_predict(scaled_unsuccessful)

# Добавление кластеров в исходные датафреймы
successful_sprints["cluster"] = successful_clusters
unsuccessful_sprints["cluster"] = unsuccessful_clusters

# Отображение обновленных данных
# import ace_tools as tools; tools.display_dataframe_to_user(name="Успешные спринты с кластерами", dataframe=successful_sprints)
# tools.display_dataframe_to_user(name="Неуспешные спринты с кластерами", dataframe=unsuccessful_sprints)


In [13]:
# 1. Расчет средних значений метрик для каждого кластера (успешные и неуспешные спринты)
successful_cluster_means = successful_sprints.groupby("cluster").mean()
unsuccessful_cluster_means = unsuccessful_sprints.groupby("cluster").mean()

# 2. Анализ вариативности метрик внутри каждого кластера
successful_cluster_variability = successful_sprints.groupby("cluster").std()
unsuccessful_cluster_variability = unsuccessful_sprints.groupby("cluster").std()

# 3. Подготовка сравнительного анализа средних значений метрик между кластерами
# Вычисляем различия между кластерами для успешных спринтов
successful_diff = successful_cluster_means.max() - successful_cluster_means.min()
unsuccessful_diff = unsuccessful_cluster_means.max() - unsuccessful_cluster_means.min()

# Создание итогового DataFrame с анализом
cluster_analysis = {
    "Successful Cluster Mean Range": successful_diff,
    "Successful Cluster Variability": successful_cluster_variability.mean(),
    "Unsuccessful Cluster Mean Range": unsuccessful_diff,
    "Unsuccessful Cluster Variability": unsuccessful_cluster_variability.mean()
}

In [15]:
# Функция для выбора только сильно выделяющихся метрик
def select_significant_metrics(metric_series, threshold=0.4):
    """
    Выбирает метрики, которые значительно отличаются от остальных.
    :param metric_series: Series с диапазонами метрик
    :param threshold: Процентное отношение между метриками, чтобы считать их выделяющимися
    :return: Список метрик
    """
    sorted_metrics = metric_series.sort_values(ascending=False)
    max_value = sorted_metrics.iloc[0]
    significant_metrics = sorted_metrics[sorted_metrics >= max_value * (1 - threshold)]
    return significant_metrics.index.tolist()

# Применяем функцию к успешным и неуспешным спринтам
successful_top_metrics_filtered = select_significant_metrics(
    cluster_analysis["Successful Cluster Mean Range"]
)
unsuccessful_top_metrics_filtered = select_significant_metrics(
    cluster_analysis["Unsuccessful Cluster Mean Range"]
)

# Формирование словаря
highlighted_metrics_filtered = {
    "successful_sprints": successful_top_metrics_filtered,
    "unsuccessful_sprints": unsuccessful_top_metrics_filtered
}

# Сохранение в JSON
# with open("highlighted_metrics_filtered.json", "w") as file:
#     json.dump(highlighted_metrics_filtered, file)

# Вывод пользователю
highlighted_metrics_filtered


{'successful_sprints': ['sprint_id', 'tasks_completed'],
 'unsuccessful_sprints': ['tasks_completed', 'sprint_id', 'team_velocity']}